# IRIS Multi Model Dimension

the following is a demo of the multi model dimension
it aims to show how you can interact with the same data
in different ways.

You will be able to see how to acces to the data in SQL and
in a pythonic way with an NoSQL approach with key-value pairs

## Import connection and utils

In [1]:
import iris
from grongier.pex._utils import iris_list_deserializer,iris_list_serializer
def run_sql(sql):
    return iris.sql.exec(sql)

## Drop the table if it exists

In [2]:
run_sql("""
DROP TABLE IF EXISTS Sample.Users
""" )

## Create the table

In [3]:
run_sql("""
CREATE TABLE Sample.Users (

         Company                                  VARCHAR(50),
         DOB                                      DATE,
         Name                                     VARCHAR(4096),
         Phone                                    VARCHAR(4096),
         Title                                    VARCHAR(50)
)
WITH %CLASSPARAMETER DEFAULTGLOBAL = '^GL.Users',
        %CLASSPARAMETER USEEXTENTSET = 0
""")


<iris.%SYS.Python.SQLResultSet at 0x7faba86b1dd0>

Note that the table reference a global called GL.Users

This global is a key-value pair where the key is the id of the record
and the value is the record itself serialized with iris list

## Insert some data

In [4]:
run_sql("""INSERT INTO Sample.Users VALUES (
            'ABC',  
            DATE('2018-01-01'),
            'John Smith',
            '555-555-5555',
            'Manager'
)
""")

<iris.%SYS.Python.SQLResultSet at 0x7faba86b1d10>

## Query the data in SQL

In [5]:
rs = run_sql("""
SELECT ID,* FROM Sample.Users
""")

for r in rs:
    print(r)

['1', 'ABC', 64649, 'John Smith', '555-555-5555', 'Manager']


## Query the data in NoSQL

### First we need to get the global

In [6]:
# First we need to get the global
g = iris.gref("^GL.UsersD")

# Then we can get the data
for k,v in g.orderiter():
    print(k,iris_list_deserializer(v))

# Update the data in NoSQL
first_line = iris_list_deserializer(g[1])

# We can update the data
first_line[2] = 'John Doe'

# And save it
g[1] = iris_list_serializer(first_line)

['ABC', 64649, 'John Smith', '555-555-5555', 'Manager']
['TOTO', 64649, 'John Smith', '555-555-5555', 'Manager']


## Query it again in SQL

In [7]:
rs = run_sql("""
SELECT ID,* FROM Sample.Users
""")
for r in rs:
    print(r)

['TOTO', 64649, 'John Smith', '555-555-5555', 'Manager']


## Now let's play only with the NoSQL approach

In [ ]:
# Create a new global
g = iris.gref("^MyGlobal")

# Insert some data
g[1] = "my first line"
g[2] = "my second line"
g[1,"a"] = "my first line with a key"
g[1,"b"] = "my first line with a key"
g[2,"a"] = "my second line with a key"
g[2,"b"] = "my second line with a key"

# Query the data

print("query all the data")
for (name, id) in g.query():
    print(f"{name} -> {id}")